In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
# ha-ha classics

import numpy as np 

import pandas as pd

import matplotlib.pyplot as plt

import sklearn

import os 

# sklearn

from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import precision_recall_curve, roc_curve

# 2nd category

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier as lgbm

# 3rd category

import torch

import torch.nn as nn
import torch.optim as optim

# misc
from tqdm import tqdm



#### Data Load


In [28]:
train = pd.read_csv('data/train_values (1).csv')
labels = pd.read_csv('data/train_labels (1).csv')
test = pd.read_csv('data/test_values (1).csv')

#### Preprocessing

In [29]:
b_ids = train.columns[0]
geo_levels = train.columns[1:4]
numeric = list(train.columns[4:8])
categorical = list(train.columns[8:15])
categorical.append(train.columns[26])
flags = train.columns[15:]

In [5]:
b_ids = train.columns[0]
geo_levels = train.columns[1:4]
numeric = train.columns[4:8]
categorical = list(train.columns[8:15])
categorical.append(train.columns[26])
print(categorical)
flags = train.columns[15:]

['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']


In [6]:
def target_encode(data,column,labels):
    mapping = {}
    
    for value in data[column].unique():
        #print(data[column])
        mask = np.array(data[column] == value)
        print('Value encoding: {}, \t mask shape: {}'.format(value, mask.shape))
        labels_mean = (labels['damage_grade'][mask]).mean()
        mapping[value] = labels_mean
    print('mapping constructed.')
    #print(mapping)
    print('processing data...')
    data[column] = data[column].apply(lambda x: mapping[x])
    
    return mapping,data
    
def target_encode_test(data_test,column,mapping):
    data_test[column] = data_test[column].apply(lambda x: mapping[x])

In [7]:
for columns in categorical:
    mapping,train = target_encode(train,columns,labels)
    #print(train[for_target])
    target_encode_test(test,columns,mapping)
    #print(test[for_target])

Value encoding: t, 	 mask shape: (260601,)
Value encoding: o, 	 mask shape: (260601,)
Value encoding: n, 	 mask shape: (260601,)
mapping constructed.
processing data...
Value encoding: r, 	 mask shape: (260601,)
Value encoding: w, 	 mask shape: (260601,)
Value encoding: i, 	 mask shape: (260601,)
Value encoding: u, 	 mask shape: (260601,)
Value encoding: h, 	 mask shape: (260601,)
mapping constructed.
processing data...
Value encoding: n, 	 mask shape: (260601,)
Value encoding: q, 	 mask shape: (260601,)
Value encoding: x, 	 mask shape: (260601,)
mapping constructed.
processing data...
Value encoding: f, 	 mask shape: (260601,)
Value encoding: x, 	 mask shape: (260601,)
Value encoding: v, 	 mask shape: (260601,)
Value encoding: z, 	 mask shape: (260601,)
Value encoding: m, 	 mask shape: (260601,)
mapping constructed.
processing data...
Value encoding: q, 	 mask shape: (260601,)
Value encoding: x, 	 mask shape: (260601,)
Value encoding: j, 	 mask shape: (260601,)
Value encoding: s, 	 ma

In [30]:
def fe(dataset):
    '''
    dataset - supposed to be train or test pd.DataFrames
    
    '''
    
    dataset['height_area_ratio'] = dataset['height_percentage']/dataset['area_percentage']
    dataset['is_old'] = dataset['age']>20
    dataset['age_sq'] = dataset['age']**2
    
fe(train)
fe(test)

numeric.extend(['height_area_ratio','is_old','age_sq'])

#### K-means clustering for geolevels

In [31]:
enc_cols = ['cluster_' + str(i) for i in range(20)]
print(enc_cols)

def append_clusters_to_dataset(main_set, clustering):
    enc = OneHotEncoder()
    ohe = enc.fit_transform(clustering.reshape(-1,1))
    ohe = ohe.toarray()
    add_on = pd.DataFrame(ohe,columns = enc_cols)
    
    X = pd.concat([main_set,add_on],axis = 1)
    return X

['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9', 'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14', 'cluster_15', 'cluster_16', 'cluster_17', 'cluster_18', 'cluster_19']


In [32]:
from sklearn.cluster import KMeans

def create_clustering_data(dataset): #instance of np.ndarray
    subdata = dataset[:,1:4].copy()
    return subdata

def perform_clustering(subdata,col_weights = [100,10,1]):
    for i in range(len(col_weights)):
        subdata[:,i] *= col_weights[i]
        
    #print(subdata.shape)
    print('...clustering in process...')
    clustering = KMeans(20).fit(subdata)
    print('...clustering done.')
    #print(clustering.labels_.shape)
    
    return clustering.labels_


In [33]:
'''
sub_train = create_clustering_data(np.array(train))
clustering_train = perform_clustering(sub_train)

train = append_clusters_to_dataset(train, clustering_train)

#### Processing check

sub_test = create_clustering_data(np.array(test))
clustering_test = perform_clustering(sub_test)

test = append_clusters_to_dataset(test, clustering_test)
print(test.shape)
'''

'\nsub_train = create_clustering_data(np.array(train))\nclustering_train = perform_clustering(sub_train)\n\ntrain = append_clusters_to_dataset(train, clustering_train)\n\n#### Processing check\n\nsub_test = create_clustering_data(np.array(test))\nclustering_test = perform_clustering(sub_test)\n\ntest = append_clusters_to_dataset(test, clustering_test)\nprint(test.shape)\n'

#### StandartScaling

In [34]:
enc = StandardScaler()

train[numeric] = enc.fit_transform(train[numeric])
test[numeric] = enc.fit_transform(test[numeric])

In [35]:
'''
def ohe(data,categorical):
    
    enc = OneHotEncoder()
    one_hot = enc.fit_transform(data[categorical]).toarray()
    #print(one_hot.shape)
    
    cols = categorical
    #print(cols)
    
    cats = enc.categories_
    #print(cats[0])

    new_column_names = []
    for k in range(len(cols)):
        for levels in range(len(cats[k])):
            #print()
            new_column_names.append(cols[k] + '_' + cats[k][levels])
            
    #print(new_column_names)     
    
    categorical_dataframe = pd.DataFrame(one_hot, columns = new_column_names)
    new_dataset = pd.concat([data.drop(categorical,axis = 1),categorical_dataframe],axis = 1)
    return new_dataset

train = ohe(train,categorical)
test = ohe(test,categorical)
'''

In [36]:
def drop_geo(dataset):
    for cols in dataset.columns:
        if 'geo' in cols:
            dataset.drop(cols,inplace = True,axis = 1)
#drop_geo(train)
#drop_geo(test)

In [37]:
building_ids = test['building_id'].copy()
train.drop('building_id',axis = 1,inplace = True)
test.drop('building_id',axis = 1,inplace = True)

In [38]:
train = np.array(train)
test = np.array(test)

#### Train-test-split

In [39]:
y = np.array(labels['damage_grade']).ravel()
np.unique(y)

array([1, 2, 3], dtype=int64)

In [55]:
X_train,X_test,y_train,y_test = train_test_split(train,
                                                 y,
                                                 random_state = 1003,
                                                test_size = 0.1)

#### Model

In [64]:
from xgboost import XGBClassifier
import lightgbm as lgbm

In [107]:
xgb = lgbm.LGBMClassifier(n_estimators = 100,
                    objective = 'multi_logloss',
                    learning_rate = 0.6,
                    verbosity = 1,
                    booster = 'gbtree',
                    n_jobs = 4,
                    subsample = 0.6
                    )

xgb.fit(train,
        y,
        early_stopping_rounds=20,
        verbose = 50,
        eval_set = [(X_test, y_test)],
        eval_metric = "multi_logloss"
       )

Training until validation scores don't improve for 20 rounds
[50]	valid_0's multi_logloss: 0.597118
[100]	valid_0's multi_logloss: 0.562885
[150]	valid_0's multi_logloss: 0.539178
[200]	valid_0's multi_logloss: 0.521119
[250]	valid_0's multi_logloss: 0.504924
[300]	valid_0's multi_logloss: 0.490071
Did not meet early stopping. Best iteration is:
[298]	valid_0's multi_logloss: 0.486654


LGBMClassifier(booster='gbtree', learning_rate=0.6, n_estimators=300, n_jobs=4,
               objective='multi_logloss', subsample=0.7, verbosity=1)

In [108]:
y_pred = xgb.predict(X_test)

In [109]:
from sklearn.metrics import f1_score

print('f1: {}'.format(f1_score(y_pred,y_test,average='micro')))

f1: 0.7928322013737001


#### Outputting

In [110]:
y_all_pred = xgb.predict(np.array(test))

In [111]:
y_all_pred.shape

(86868,)

In [112]:
test_r = pd.read_csv('data/test_values (1).csv')
building_ids = test_r['building_id']

In [113]:
def create_submission(y_all_pred,building_ids,name):

    preds = y_all_pred
    sub = pd.DataFrame()
    sub['building_id'] = building_ids
    sub['damage_grade'] = y_all_pred
    sub.set_index('building_id',inplace = True)
    print(sub.shape)
    sub.to_csv('subs/' + name + '.csv')

In [114]:
name = 'sub_xgb_mix_model3'
create_submission(y_all_pred,building_ids,name)

(86868, 1)


#### Fun

In [115]:
m1 = pd.read_csv('subs/sub_xgb_mix_model1.csv')
m2 = pd.read_csv('subs/sub_xgb_mix_model2.csv')
m3 = pd.read_csv('subs/sub_xgb_mix_model3.csv')

In [116]:
pred1 = m1['damage_grade']
pred2 = m2['damage_grade']
pred3 = m3['damage_grade']

In [120]:
output = np.median([pred1,pred2,pred3],axis = 0)

In [123]:
(output == pred3).sum()/len(output)

0.9575217571487774

In [129]:
y_all_pred = output.astype(np.int64)

In [130]:
y_all_pred.shape

(86868,)

In [131]:
test_r = pd.read_csv('data/test_values (1).csv')
building_ids = test_r['building_id']

In [132]:
def create_submission(y_all_pred,building_ids,name):

    preds = y_all_pred
    sub = pd.DataFrame()
    sub['building_id'] = building_ids
    sub['damage_grade'] = y_all_pred
    sub.set_index('building_id',inplace = True)
    print(sub.shape)
    sub.to_csv('subs/' + name + '.csv')

In [133]:
name = 'sub_xgb_mix_output'
create_submission(y_all_pred,building_ids,name)

(86868, 1)


In [231]:
import catboost

#### Structured Fun

In [287]:
model_outputs = {}

n = 7
model_seeds = np.random.randint(1,10000,n)
n_estimatorss = np.random.randint(40,80,n)
learning_rates = np.random.uniform(0,1,n)

for iters in tqdm(range(n)):
    
    xgb = XGBClassifier(n_estimators = n_estimatorss[iters], #
                    objective = 'mse',
                    learning_rate = learning_rates[iters],
                    max_depth = 13,
                    colsample_bytree=0.75,
                    verbosity = 1,
                    booster = 'gbtree',
                    n_jobs = 4,
                    subsample = 0.75,
                    random_state = model_seeds[iters],
                    
                    )

    xgb.fit(train,
            y,
            early_stopping_rounds=20,
            verbose = 10,
            eval_set = [(train,y)],
            eval_metric = "mlogloss"
           )
    
    y_all_pred = xgb.predict(np.array(test))
    
    
    
    model_outputs[iters] = y_all_pred

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

[0]	validation_0-mlogloss:0.83093
Will train until validation_0-mlogloss hasn't improved in 20 rounds.
[10]	validation_0-mlogloss:0.47177
[20]	validation_0-mlogloss:0.41560
[30]	validation_0-mlogloss:0.36628
[40]	validation_0-mlogloss:0.32601
[50]	validation_0-mlogloss:0.29168
[60]	validation_0-mlogloss:0.26412
[64]	validation_0-mlogloss:0.25316


 14%|███████████▊                                                                       | 1/7 [08:38<51:52, 518.68s/it]

[0]	validation_0-mlogloss:1.02055
Will train until validation_0-mlogloss hasn't improved in 20 rounds.
[10]	validation_0-mlogloss:0.65894
[20]	validation_0-mlogloss:0.55627
[30]	validation_0-mlogloss:0.50941
[40]	validation_0-mlogloss:0.48128
[47]	validation_0-mlogloss:0.46634


 29%|███████████████████████▋                                                           | 2/7 [16:29<42:01, 504.28s/it]

...

 71%|███████████████████████████████████████████████████████████▎                       | 5/7 [42:22<15:49, 474.83s/it]

[0]	validation_0-mlogloss:0.83728
Will train until validation_0-mlogloss hasn't improved in 20 rounds.
[10]	validation_0-mlogloss:0.50402
[20]	validation_0-mlogloss:0.43541
[30]	validation_0-mlogloss:0.39514
[40]	validation_0-mlogloss:0.36230
[50]	validation_0-mlogloss:0.33095
[60]	validation_0-mlogloss:0.30147
[70]	validation_0-mlogloss:0.27669
[71]	validation_0-mlogloss:0.27465


 86%|███████████████████████████████████████████████████████████████████████▏           | 6/7 [44:51<06:16, 376.87s/it]

[0]	validation_0-mlogloss:0.78076
Will train until validation_0-mlogloss hasn't improved in 20 rounds.
[10]	validation_0-mlogloss:0.46131
[20]	validation_0-mlogloss:0.39750
[30]	validation_0-mlogloss:0.35056
[40]	validation_0-mlogloss:0.30610
[50]	validation_0-mlogloss:0.27014
[60]	validation_0-mlogloss:0.24173
[70]	validation_0-mlogloss:0.21568
[76]	validation_0-mlogloss:0.20239


100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [47:37<00:00, 408.20s/it]


In [288]:
output = np.median(np.concatenate(list(model_outputs.values())).reshape(n,-1),axis = 0)
output.shape

(86868,)

In [289]:
y_all_pred = output.astype(np.int64)

In [291]:
#print(f1_score(y_all_pred,y_test,average = 'micro'))

In [293]:
test_r =pd.read_csv('data/test_values (1).csv')
building_ids = test_r['building_id']

In [294]:
def create_submission(y_all_pred,building_ids,name):

    preds = y_all_pred
    sub = pd.DataFrame()
    sub['building_id'] = building_ids
    sub['damage_grade'] = y_all_pred
    sub.set_index('building_id',inplace = True)
    print(sub.shape)
    sub.to_csv('subs/' + name + '.csv')

In [295]:
name = '8_xgb'
create_submission(y_all_pred,building_ids,name)

(86868, 1)
